In [16]:
#for tutorial usage only. this is not part of our project
import folium

from folium import Map
from pydantic import BaseModel


class Point(BaseModel):
    lat: float
    longg: float

def get_folium_map(center: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(location=[center.lat, center.longg], zoom_start=zoom_level)

    for point in points:
        folium.Marker(location=[point.lat, point.longg], popup='Point').add_to(folium_map)
    return folium_map

#examples 
# no1 13.174019, 123.750013 -> Bahay ko lmao
point_1 = Point(lat=13.173991938817364, longg=123.74993419280142)
# no2 13.144499960685334, 123.7226020150628 -> bu main
point_2 = Point(lat=13.144499960685334, longg=123.7226020150628)
#center point -> 13.160920164457547, 123.73989775927346 (basta leg toh, gitna ng dalawa)
center_point = Point(lat=13.160920164457547, longg=123.73989775927346)

folium_map = get_folium_map(center_point, [point_1, point_2])
folium_map 

In [ ]:
service = 'route'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

In [18]:
import requests

points = [point_1, point_2]
coordinates = ';'.join(
    [f'{point.longg},{point.lat}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{
    coordinates}?overview=full&steps=true&geometries=geojson"

response = requests.get(url)
response.status_code

200

In [19]:
data = response.json()
data.keys()

dict_keys(['code', 'routes', 'waypoints'])

In [20]:
routes = data['routes']
print(f"Found {len(routes)} route(s)")

Found 1 route(s)


In [21]:
route = routes[0]
route_coordinates = route['geometry']["coordinates"]
route_distance = route["distance"]  # unit: meters
route_duration = route["duration"]  # unit: seconds

route_coordinates[:5]

[[123.749932, 13.174012],
 [123.749982, 13.174016],
 [123.750171, 13.174032],
 [123.750201, 13.174038],
 [123.751002, 13.174128]]

In [22]:
route_coordinates = [[point[1], point[0]] for point in route_coordinates]
route_coordinates[:5]

[[13.174012, 123.749932],
 [13.174016, 123.749982],
 [13.174032, 123.750171],
 [13.174038, 123.750201],
 [13.174128, 123.751002]]

In [23]:
folium_map = get_folium_map(center_point, [point_1, point_2])
popup_text = f"Distance: {route_distance} meters<br>Duration: {
    route_duration} seconds"
popup = folium.Popup(popup_text, max_width=300)

folium.PolyLine(
    locations=route_coordinates, color='blue', weight=5, popup=popup).add_to(folium_map)

folium_map

In [24]:
folium_map = get_folium_map(center_point, [point_1, point_2])

colors = ['black', 'blue']

legs = data["routes"][0]["legs"]
for i, leg in enumerate(legs):
    steps = leg["steps"]
    for step in steps:
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]
        popup_text = f"""
        Leg number {i + 1}<br>
        Leg distance: {leg['distance']} meters<br>
        Leg duration: {leg['duration']} seconds<br>
        """
        popup = folium.Popup(popup_text, max_width=300)
        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup)
        folium_map.add_child(polyline)

folium_map

In [25]:
folium_map = get_folium_map(
    point_1, [point_1, point_2], zoom_level=16)

colors = ['black', 'blue']

legs = data["routes"][0]["legs"]
for leg in legs:
    steps = leg["steps"]
    for i, step in enumerate(steps[:2]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]
        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)
        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup)
        folium_map.add_child(polyline)
    break

folium_map

KeyError: 'modifier'

In [26]:
from pprint import pprint

leg = data["routes"][0]["legs"][0]
step = leg["steps"][0]
pprint(step)

{'distance': 223.1,
 'driving_side': 'right',
 'duration': 34.9,
 'geometry': {'coordinates': [[123.749932, 13.174012],
                              [123.749982, 13.174016],
                              [123.750171, 13.174032],
                              [123.750201, 13.174038],
                              [123.751002, 13.174128],
                              [123.751105, 13.174142],
                              [123.751375, 13.174168],
                              [123.751473, 13.174178],
                              [123.75198, 13.174195]],
              'type': 'LineString'},
 'intersections': [{'bearings': [85],
                    'entry': [True],
                    'location': [123.749932, 13.174012],
                    'out': 0},
                   {'bearings': [90, 180, 270],
                    'entry': [True, True, False],
                    'in': 2,
                    'location': [123.749982, 13.174016],
                    'out': 0},
                   {'bear

In [27]:
import math


def add_compass(folium_map: Map, location: tuple, bearing: int, size: int = 50):

    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: red;">📍</div>"""),
        tooltip=f"Bearing: {bearing}°",
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color="red",
        weight=2,
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2], zoom_level=16)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        maneuver_location = step["maneuver"]["location"]
        bearing = step["maneuver"].get("bearing_after", 0)
        add_compass(
            folium_map=folium_map,
            location=(maneuver_location[1], maneuver_location[0]),
            bearing=bearing
        )

folium_map

KeyError: 'modifier'

In [ ]:
from pprint import pprint

leg = data["routes"][0]["legs"][0]
step = leg["steps"][0]
intersections = step["intersections"]
pprint(intersections)

In [ ]:
import math


def add_compass(
    folium_map: Map,
    location: tuple,
    bearing: int,
    size: int = 50,
    color: str = "red",
    weight: int = 2
):
    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: red;">📍</div>"""),
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color=color,
        weight=weight,
        tooltip=f"Bearing: {bearing}°",
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2], zoom_level=18)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        intersections = step["intersections"]
        for intersection in intersections[:1]:
            location = intersection["location"]
            bearings = intersection["bearings"]
            for j, bearing in enumerate(bearings):
                allowed_for_traversal = intersection["entry"][j]
                add_compass(
                    folium_map=folium_map,
                    location=(location[1], location[0]),
                    bearing=bearing,
                    color="green" if allowed_for_traversal else "red",
                    weight=5
                )

folium_map

In [ ]:
import math


def add_compass(
    folium_map: Map,
    location: tuple,
    bearing: int,
    size: int = 50,
    color: str = "red",
    weight: int = 2
):
    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: black;">📍</div>"""),
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color=color,
        weight=weight,
        tooltip=f"Bearing: {bearing}°"
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2], zoom_level=20)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        intersections = step["intersections"]
        for intersection in intersections[1:]:
            location = intersection["location"]
            bearings = intersection["bearings"]
            for j, bearing in enumerate(bearings):
                allowed_for_traversal = intersection["entry"][j]
                add_compass(
                    folium_map=folium_map,
                    location=(location[1], location[0]),
                    bearing=bearing,
                    color="green" if allowed_for_traversal else "red",
                    weight=5
                )

folium_map

In [ ]:
weight = route["weight"]
weight_name = route["weight_name"]
duration = route["duration"]
distance = route["distance"]

print(f"Weight: {weight} - {weight_name}")
print(f"Duration: {duration} seconds")
print(f"Distance: {distance} meters")

In [ ]:
import requests

points = [point_1, point_2]
coordinates = ';'.join(
    [f'{point.longg},{point.lat}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{
    coordinates}?alternatives=3&overview=full&steps=true&geometries=geojson"

response = requests.get(url)
data = response.json()
routes = data['routes']
print(f"Found {len(routes)} route(s)")